# 509 Final Project

The notebook is for text data preprocessing.

## Globally import libraries

Libraries needed mostly pertain to dataframe manipulation for data preprocessing.

In [1]:
from collections import defaultdict, Counter
import datetime as dt
import emoji
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import regex as rex
import requests
import shutil
from string import punctuation
import time
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from sklearn.feature_extraction.text import TfidfTransformer, \
CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import metrics
from sklearn.metrics import make_scorer, f1_score, classification_report, \
confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay

import textacy.preprocessing as tprep
from textacy.extract import keyword_in_context

# Set pandas global options
pd.options.display.max_rows = 17
pd.options.display.precision = 4
np.set_printoptions(suppress=True, precision=4)

%matplotlib inline

## Upload data from CSV

Establish working directories for saving dataframes as CSV files

In [2]:
'''Dir nav citation:
https://softhints.com/python-change-directory-parent/'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\599_team_project\deliverables
C:\Users\acarr\Documents\GitHub\599_team_project


Get current date/time to append to file name string

In [3]:
today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
print(today)
print(type(today))

2023-07-14_20-17-09395488
<class 'str'>


Establish full file name path

In [4]:
# change `data_location` to the location of the folder on your machine.
data_location = 'data'

file_in_name01 = 'data_scraped_amc_2023-07-10_13-41-28670390.csv'

file_in_path01 = os.path.join(up1_dir, data_location, file_in_name01)

print(f'CSV file in 1 path: {file_in_path01}')

CSV file in 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data\data_scraped_amc_2023-07-10_13-41-28670390.csv


### Review dataframe

Read in data from CSV, check resulting dataframe shape, and display first several records.

In [5]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)
print(f'Dataframe shape: {slct_tbl_full_df01.shape}')
display(slct_tbl_full_df01.head())

Dataframe shape: (36420, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...


## Initial (prior to preprocessing) Exploratory Data Analysis (EDA)

Perform only the EDA steps needed to preprocess the data, e.g., checking for missing values.

### Count missing `article_text` feature

The majority of null values appear in the `author` column. There are also two in `article_text`. The `author` feature will not be used for current modeling efforts, therefore it is not a factor. The instances with missing article text will be removed.

In [6]:
count_nan = slct_tbl_full_df01.isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: text_id           0
source_name       0
author          216
title             0
url               0
publish_date      0
article_text      2
content           0
dtype: int64


### Count blank `article_text` feature

Show instances with blank or missing `article_text` values; *n* = 2

In [7]:
print(len(slct_tbl_full_df01[slct_tbl_full_df01['article_text']==None]))
display(slct_tbl_full_df01[slct_tbl_full_df01['article_text'].isna()].head(20))

0


,text_id,source_name,author,title,url,publish_date,article_text,content
20850,22986,Fox News,Fox News Staff,"Mug shots of the week: June 11-17, 2023",https://www.foxnews.com/us/mug-shots-week-june...,2023-06-18T00:00:51Z,NaN,These mug shots were taken for arrests made th...
34174,66130,Fox News,Fox News Staff,"Mug shots of the week: June 18-24, 2023",https://www.foxnews.com/us/mug-shots-week-june...,2023-06-25T00:00:48Z,NaN,These mug shots were taken for arrests made th...


### Remove missing `article_text` row(s)

Use pandas' `dropna()` method to remove instances with null values; send results to a new dataframe. Print resulting df shape and head.

In [8]:
'''Drop missing citation:
https://pandas.pydata.org/pandas-docs/stable/reference
/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna'''
slct_tbl_full_df02 = slct_tbl_full_df01.dropna(subset=['article_text'])
print(f'Dataframe shape: {slct_tbl_full_df02.shape}')
display(slct_tbl_full_df02.head())

Dataframe shape: (36418, 8)


,text_id,source_name,author,title,url,publish_date,article_text,content
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...


Set tqdm package progress bar.

In [9]:
tqdm.pandas(ncols=50)  # can use tqdm_gui, optional kwargs, etc
# Now you can use `progress_apply` instead of `apply`

### Display Source counts

Count the number of instances within each news source, e.g., ABC, CNN, USA Today.

In [10]:
display(slct_tbl_full_df02['source_name'].value_counts().to_string())

'Forbes                 7786\nNew York Post          6862\nUSA Today              6752\nABC News               4858\nCNN                    1686\nBusiness Insider       1563\nFox News               1379\nCNBC                   1338\nNBC News               1111\nBreitbart News         1038\nBuzzfeed                750\nThe Washington Post     374\nPEOPLE                  245\nMSNBC                   225\nVox                     213\nWired                   129\nReuters                  96\nmsnNOW                   13'

### Display Author counts

Display the count of `author` values.

## Data preprocessing

Define a function to display all individual tokens across the entire dataset.

In [11]:
def uniq_tok(df_col=pd.DataFrame([''])):
    '''Display all unique tokens across all instances'''
    df_cols1 = pd.Series(df_col)

    all_tokens_lst01 = []

    [all_tokens_lst01.append(f) for f in df_cols1]
    all_tokens_lst01 = list(itertools.chain.from_iterable(all_tokens_lst01))
    all_tokens_set01 = set(all_tokens_lst01)
    print(len(sorted(all_tokens_set01)))
    print(sorted(all_tokens_set01))
    return sorted(all_tokens_set01)

Create fresh dataframes for different purposes.

In [12]:
slct_tbl_full_df03 = slct_tbl_full_df02.copy()
slct_tbl_full_df04 = slct_tbl_full_df02.copy()
slct_tbl_full_df05 = slct_tbl_full_df02.copy()

In [13]:
source_name_lst = slct_tbl_full_df03['source_name'].unique()
print(source_name_lst)

['USA Today' 'ABC News' 'CNN' 'Business Insider' 'Breitbart News' 'MSNBC'
 'Buzzfeed' 'Wired' 'NBC News' 'The Washington Post' 'Fox News' 'Reuters'
 'New York Post' 'Forbes' 'PEOPLE' 'CNBC' 'Vox' 'msnNOW']


### Case-loading

Apply the `str.lower()` function to entire `article_text` column of the dataframe.

In [14]:
slct_tbl_full_df03['lower'] = slct_tbl_full_df03['article_text']\
.apply(str.lower)

'''
Remove float citation:
https://chat.openai.com/share/dc596310-0c93-4542-9064-3034e85ed0a1
'''

slct_tbl_full_df03['author_source'] = slct_tbl_full_df03['source_name'] + ' ' + slct_tbl_full_df03['author']
slct_tbl_full_df03['author_lower'] = slct_tbl_full_df03['author_source']\
.apply(lambda x: str(x).lower() if isinstance(x, str) else '')

def clean_author(text):
    text_p = text.replace(',', ' ')
    text_p = text_p.split(' ')
    #text_p = list(text_p)
    text_p = set(text_p)
    text_p = list(text_p)
    return text_p

slct_tbl_full_df03['author_lower_p'] = slct_tbl_full_df03['author_lower']\
.apply(clean_author)
slct_tbl_full_df03['author_lower_p_join'] = slct_tbl_full_df03['author_lower_p'].apply(" ".join)

Print shape and first few records of resulting dataframe.

In [15]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

(36418, 13)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,author_lower,author_lower_p,author_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[, deletter, usa, today, emily]",deletter usa today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[, usa, ashford, today, joy]",usa ashford today joy
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, today, wire, tar, heels]",usa today wire tar heels
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[roll, usa, today, wire, tide]",roll usa today wire tide
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, today, celtics]",usa wire today celtics


In [16]:
uniq_tok(slct_tbl_full_df03['author_lower_p'])

10117
['', '\n', '"jay"', '"toby"', '&', "'smitty'", '(aaron', '(al', '(alcynna', '(alex', '(alexa', '(alexander', '(alexandra', '(alia', '(alistair', '(angela', '(antonio', '(ashley', '(associated', '(ayelet', '(azmi', '(beatrice', '(ben', '(bethany', '(bianca', '(brendan', '(brent', '(bryan', '(cadie', '(charles', '(cheryl', '(chong)', '(chris', '(christopher', '(cork', '(dan', '(diamond', '(dominick', '(elias', '(eliza', '(ella', '(ellen', '(emily', '(erin', '(filip', '(george', '(gia', '(grace', '(graham', '(hallam', '(hannah', '(hasan', '(haven', '(helaine', '(huileng', '(indy)', '(insider', '(isobel', '(jack', '(jackson', '(jacob', '(jake', '(jason', '(jeffrey', '(jennifer', '(jenny', '(john', '(jordan', '(josée', '(juliana', '(jyoti', '(kaila', '(katherine', '(katie', '(kelly', '(kelsey', '(kenneth', '(kieran', '(kimberly', '(kwan', '(lakshmi', '(lauren', '(leslie', '(lily', '(lina', '(lloyd', '(lydia', '(madison', '(marianne', '(marie', '(mariette', '(matt', '(matthew', '(max',

['',
 '\n',
 '"jay"',
 '"toby"',
 '&',
 "'smitty'",
 '(aaron',
 '(al',
 '(alcynna',
 '(alex',
 '(alexa',
 '(alexander',
 '(alexandra',
 '(alia',
 '(alistair',
 '(angela',
 '(antonio',
 '(ashley',
 '(associated',
 '(ayelet',
 '(azmi',
 '(beatrice',
 '(ben',
 '(bethany',
 '(bianca',
 '(brendan',
 '(brent',
 '(bryan',
 '(cadie',
 '(charles',
 '(cheryl',
 '(chong)',
 '(chris',
 '(christopher',
 '(cork',
 '(dan',
 '(diamond',
 '(dominick',
 '(elias',
 '(eliza',
 '(ella',
 '(ellen',
 '(emily',
 '(erin',
 '(filip',
 '(george',
 '(gia',
 '(grace',
 '(graham',
 '(hallam',
 '(hannah',
 '(hasan',
 '(haven',
 '(helaine',
 '(huileng',
 '(indy)',
 '(insider',
 '(isobel',
 '(jack',
 '(jackson',
 '(jacob',
 '(jake',
 '(jason',
 '(jeffrey',
 '(jennifer',
 '(jenny',
 '(john',
 '(jordan',
 '(josée',
 '(juliana',
 '(jyoti',
 '(kaila',
 '(katherine',
 '(katie',
 '(kelly',
 '(kelsey',
 '(kenneth',
 '(kieran',
 '(kimberly',
 '(kwan',
 '(lakshmi',
 '(lauren',
 '(leslie',
 '(lily',
 '(lina',
 '(lloyd',
 '(lydi

### Text normalization

#### Create function

Define function to normalize aspects of the text, including hyphens, quotation marks, and unicode; remove accent marks, where applicable.

In [17]:
def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text

#### Call function

Apply custom function to entire text column of the dataframe.

In [18]:
slct_tbl_full_df03['norm'] = slct_tbl_full_df03['lower'].apply(normalize)
slct_tbl_full_df03['norm_author_lower_p_join'] = slct_tbl_full_df03['author_lower_p_join'].apply(normalize)

Print shape and first few records of resulting dataframe; also display a sample of full text.

In [19]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,4):
    try:
        display(slct_tbl_full_df03['norm'][c], '\n')
    except:
        print(f'Skip {c}')

(36418, 15)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,author_lower,author_lower_p,author_lower_p_join,norm,norm_author_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[, deletter, usa, today, emily]",deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[, usa, ashford, today, joy]",usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, today, wire, tar, heels]",usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[roll, usa, today, wire, tide]",roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, today, celtics]",usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics


'have you ever wanted to own your very own keg of tito\'s handmade vodka? now, that dream can become a reality.the vodka brand is launching tito\'s in a big can, a $200, 128-ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co2 regulator, stainless steel spear, tap and handle and an extra set of o-rings. it comes empty, but tito\'s said it can be filled with their own vodka or with anything else.tito\'s is also releasing tito\'s in any can, a smaller insulated can-cooler for $30 that also functions as a refillable cup. it can fit both 16-ounce and 12-ounce cans, and converts into a refillable pint tumbler with a leak-proof lid."rebel against canformity," the product\'s description reads. "because we don\'t make canned cocktails, pre-mixes, or seltzers. you do."10best:which fast food restaurants will reign supreme? cast your votes nowwhat are americans drinking this summer?tequila, modelo, canned cocktails and non-alcoholic drinksboth products are available fo

'\n'

'over five months after julian sands went missing during a solo hike in california, local police have announced the ongoing search for the actor will continue in a "limited capacity."the san bernardino county sheriff\'s office released a statement monday on the department\'s continued efforts to locate the "a room with a view" actor.sands, 65, was reported missing on jan. 13 while hiking a trail on mount baldy in southern california, san bernardino county sheriff\'s department spokeswoman gloria huerta said at the time. authorities\' search efforts, both on the ground and by air, have been paused and resumed based on a series of storms that has created dangerous mountain conditions in the area.on saturday, the sheriff\'s office conducted another search for sands involving over 80 search and rescue volunteers, deputies and staff. two helicopters helped search teams reach remote areas of mount baldy, and drone crews searched areas inaccessible to hikers on the ground.despite the departme

'\n'

'\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running back picks michigan state over unc\n\n\n\n\t\t\tshare this article\t\t\n\n\t\t\t\t35 shares\n\t\t\n\n\n \n\t\t\t\t\tshare\t\t\t\t\n\n\n\n \n\t\t\t\t\ttweet\t\t\t\t\n\n\n\n \n\t\t\t\ttext\t\t\t\n\n\n\n \n\t\t\t\temail\t\t\t\n\n\n\n \n\t\t\t\tlink\t\t\t\n\n\n\n \n\n\n\n\n \n\t\t\t\ton facebook\t\t\t\n\n\n\n \n\t\t\t\ton twitter\t\t\t\n\n\n\n \n\t\t\t\tvia text message\t\t\t\n\n\n\n \n\t\t\t\ton linkedin\t\t\t\n\n\n\n \n\t\t\t\tvia email\t\t\t\n\n\n\n \n\t\t\t\thttps://tarheelswire.usatoday.com/2023/06/24/four-star-running-back-picks-michigan-state-over-unc/\t\t\t\n\n\n\n\n\n\n\n\n\n\nzack pearson \n\n\n\njune 24, 2023 11:51 pm et\n\n\n\n\n\n\n\n\nmack brown and the unc football program have been on a bit of a hot streak lately in terms of landing commitments in the 2024 recruiting class.\nbut on saturday, they couldn\'t keep that good luck rolling.\nbuy tar heels tickets\nwith the chance to add their second four-star running back of the 2024

'\n'

"\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center charles bediako signs one-year deal with san antonio spurs\n\n\n\n\t\t\tshare this article\t\t\n\n\t\t\t\t92 shares\n\t\t\n\n\n \n\t\t\t\t\tshare\t\t\t\t\n\n\n\n \n\t\t\t\t\ttweet\t\t\t\t\n\n\n\n \n\t\t\t\ttext\t\t\t\n\n\n\n \n\t\t\t\temail\t\t\t\n\n\n\n \n\t\t\t\tlink\t\t\t\n\n\n\n \n\n\n\n\n \n\t\t\t\ton facebook\t\t\t\n\n\n\n \n\t\t\t\ton twitter\t\t\t\n\n\n\n \n\t\t\t\tvia text message\t\t\t\n\n\n\n \n\t\t\t\ton linkedin\t\t\t\n\n\n\n \n\t\t\t\tvia email\t\t\t\n\n\n\n \n\t\t\t\thttps://rolltidewire.usatoday.com/2023/06/23/alabama-basketball-center-charles-bediako-signs-one-year-deal-with-san-antonio-spurs/\t\t\t\n\n\n\n\n\n\nsteve roberts-usa today sports\n\n\n\n\nbrody smoot \n\n\n\njune 23, 2023 4:29 pm ct\n\n\n\n\n\n\n\n\nalabama center charles bediako was signed to a one-year deal by the san antonio spurs. bediako went undrafted in the 2023 nba draft.\nthe native of brampton, ontario spent two seasons in tuscaloosa. as a freshman, bed

'\n'

### Remove source/website specific phrases using RegEx

#### Create function

Define function to eliminate specific website artifacts that add noise to the data.

In [20]:
rex_like_article = rex.compile(r'like this article\?[\n]+sign[(up)|(for)|(the)|\s]+\s[\w\s]*newsletter to get our top stories in your inbox every morning[\n,\s,\t]')
#rex_click_here = rex.compile(r'click here to sign up for[(our)|(the)\s]+\s[\w\s]*newsletter')
rex_click_here = rex.compile(r'click(?:here|to|get|ge|the|for|sign|read|up|our|hear|more|instagram|twitter|view|learn|\s)*[\w\s]*(?:newsletter|app|reporting|cartoons){1}\b')
rex_copyright = rex.compile(r'copyright\s[\d]+[\w\s]*[^\n]+[\.|\n]{1}')
rex_sign_up = rex.compile(r'email[\n|\t|\s]+sign up[\n|\t|\s]+view all newsletters[\n|\t|\s]+more[\w\s]+[\n|\t|\s]{1}')
rex_gamble = rex.compile(r'gannett[\S|\s|]*gamble\.')
rex_tipco = rex.compile(r'catch the excitement[\S|\s|\n]+conditions.')
rex_tc = rex.compile(r'must be\s[\d]+[\S|\s]+apply\.')

def remove_rex(text):
    repl_text = rex_like_article.sub(r' ', text)
    repl_text = rex_click_here.sub(r' ', repl_text)
    repl_text = rex_copyright.sub(r' ', repl_text)
    repl_text = rex_sign_up.sub(r' ', repl_text)
    repl_text = rex_gamble.sub(r' ', repl_text)
    repl_text = rex_tipco.sub(r' ', repl_text)
    repl_text = rex_tc.sub(r' ', repl_text)
    return repl_text

#### Call function

Apply custom function to entire text column of the dataframe.

In [21]:
slct_tbl_full_df03['rex_trimmed'] = slct_tbl_full_df03['norm'].apply(remove_rex)

In [22]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

(36418, 16)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,author_lower,author_lower_p,author_lower_p_join,norm,norm_author_lower_p_join,rex_trimmed
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[, deletter, usa, today, emily]",deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[, usa, ashford, today, joy]",usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, today, wire, tar, heels]",usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[roll, usa, today, wire, tide]",roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, today, celtics]",usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...


In [23]:
for s in source_name_lst:
    df01 = slct_tbl_full_df03.loc[slct_tbl_full_df03['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['rex_trimmed'][c], '\n')
        except:
            print(f'Skip {c}')

USA Today article #1
URL: https://www.usatoday.com/story/money/food/2023/06/21/titos-vodka-keg-price/70342132007/
Author(s): USA TODAY, Emily DeLetter, USA TODAY
Title: Tito's launches 'Tito's in a Big Can,' an empty cocktail keg listed at $200
have you ever wanted to own your very own keg of tito's handmade vodka? now, that dream can become a reality.the vodka brand is launching tito's in a big can, a $200, 128-ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co2 regulator, stainless steel spear, tap and handle and an extra set of o-rings. it comes empty, but tito's said it can be filled with their own vodka or with anything else.tito's is also releasing tito's in any can, a smaller insulated can-cooler for $30 that also functions as a refillable cup. it can fit both 16-ounce and 12-ounce cans, and converts into a refillable pint tumbler with a leak-proof lid."rebel against canformity," the product's description reads. "because we don't make canned cocktail

### Remove special characters

#### Create function

In [24]:
'''re.sub lambda citation:
https://chat.openai.com/share/402ec66e-2802-4cda-af8c-6f9f5b097d85
'''
sep_lst = []
ucode_lst = []
# Add leading and trailing space to URLs
def rex_replace(text):
    text = text.replace(r'&nbsp;', ' ').replace(r'-', ' ')\
    .replace('\u2063', ' ').replace('\u2066', ' ')\
    .replace('\u2069', ' ').replace('\u200b', ' ').replace('\u200d', ' ')\
    .replace('(click to view)', ' ')\
    .replace('a post shared by', ' ')\
    .replace('app users click here', ' ')\
    .replace('app users: click here', ' ')\
    .replace('app users, click here:', ' ')\
    .replace('click here.', ' ')\
    .replace('click here for more', ' ')\
    .replace('click here for the latest fox news reporting', ' ')\
    .replace('click here for topline and cross tabs conducted', ' ')\
    .replace('click here to hear more', ' ')\
    .replace('click here to learn more', ' ')\
    .replace('click here to read more', ' ')\
    .replace('click here to subscribe and get your first year of fox nation free of charge', ' ')\
    .replace("click to get kurt's cyberguy newsletter with quick tips, tech reviews, security alerts and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's cyberguy newsletter with security alerts, quick tips, tech reviews, security and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's free cyberguy newsletter with quick tips, tech reviews, security alerts and easy how to's to make you smarter", ' ')\
    .replace("click to get kurt's free cyberguy newsletter with security alerts, quick tips, tech reviews, and easy how to's to make you smarter", ' ')\
    .replace('fox news digital', ' ')\
    .replace('request for comment', ' ')\
    .replace('the ap ', ' ')\
    .replace('copyright © 2023 breitbart', ' ')\
    .replace('all rights reserved', ' ')\
    .replace('copyright 2023 fox news network', ' ')\
    .replace('copyright 2023 viq media transcription', ' ')\
    .replace("please let us know if you're having issues with commenting", ' ')\
    .replace('view this post on instagram', ' ')
    return text

#### Call function

Apply custom function to entire text column of the dataframe.

In [25]:
slct_tbl_full_df03['trimmed'] = slct_tbl_full_df03['rex_trimmed'].apply(rex_replace)
slct_tbl_full_df03['trimmed_norm_author_lower_p_join'] = slct_tbl_full_df03['norm_author_lower_p_join'].apply(rex_replace)

In [26]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

(36418, 18)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,author_lower,author_lower_p,author_lower_p_join,norm,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[, deletter, usa, today, emily]",deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[, usa, ashford, today, joy]",usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, today, wire, tar, heels]",usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[roll, usa, today, wire, tide]",roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, today, celtics]",usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics


### Separate emojis as individual tokens

#### Create function

In [27]:
def emoji_split(text):
    return("".join([' ' + c + ' ' if emoji.is_emoji(c) else c for c in text]))

#### Call function

Apply custom function to entire text column of the dataframe.

In [28]:
slct_tbl_full_df03['emoji_split'] = slct_tbl_full_df03['trimmed']\
.apply(emoji_split)

In [29]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    try:
        print(slct_tbl_full_df03['emoji_split'][c], '\n')
    except:
        print(f'Skip {c}')

(36418, 19)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,author_lower,author_lower_p,author_lower_p_join,norm,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY","usa today usa today, emily deletter, usa today","[, deletter, usa, today, emily]",deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY","usa today usa today, joy ashford, usa today","[, usa, ashford, today, joy]",usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,usa today tar heels wire,"[usa, today, wire, tar, heels]",usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,usa today roll tide wire,"[roll, usa, today, wire, tide]",roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,usa today celtics wire,"[usa, wire, today, celtics]",usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...


have you ever wanted to own your very own keg of tito's handmade vodka? now, that dream can become a reality.the vodka brand is launching tito's in a big can, a $200, 128 ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co2 regulator, stainless steel spear, tap and handle and an extra set of o rings. it comes empty, but tito's said it can be filled with their own vodka or with anything else.tito's is also releasing tito's in any can, a smaller insulated can cooler for $30 that also functions as a refillable cup. it can fit both 16 ounce and 12 ounce cans, and converts into a refillable pint tumbler with a leak proof lid."rebel against canformity," the product's description reads. "because we don't make canned cocktails, pre mixes, or seltzers. you do."10best:which fast food restaurants will reign supreme? cast your votes nowwhat are americans drinking this summer?tequila, modelo, canned cocktails and non alcoholic drinksboth products are available for a limi

### Remove URLs

#### Create function

In [30]:
#rex_url_c = rex.compile(r'http[s]?:[\/]+[\S]*\s')
rex_url_c = rex.compile(r'(?:http|https|www)+[\.|\S]+\b')

'''re.sub lambda citation:
https://chat.openai.com/share/402ec66e-2802-4cda-af8c-6f9f5b097d85
'''
# Add leading and trailing space to URLs
def remove_url(text):
    text = rex_url_c.sub(r' ', text)
    return text

#### Call function

Apply custom function to entire text column of the dataframe.

In [31]:
slct_tbl_full_df03['urls_removed'] = slct_tbl_full_df03['emoji_split'].apply(remove_url)

slct_tbl_full_df03['urls_removed_norm_author_lower_p_join'] = slct_tbl_full_df03['trimmed_norm_author_lower_p_join'].apply(remove_url)

In [32]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

(36418, 21)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,author_lower_p,author_lower_p_join,norm,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,"[, deletter, usa, today, emily]",deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,"[, usa, ashford, today, joy]",usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,"[usa, today, wire, tar, heels]",usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,"[roll, usa, today, wire, tide]",roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,"[usa, wire, today, celtics]",usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics


In [33]:
for s in source_name_lst:
    df01 = slct_tbl_full_df03.loc[slct_tbl_full_df03['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['urls_removed'][c], '\n')
        except:
            print(f'Skip {c}')

USA Today article #1
URL: https://www.usatoday.com/story/money/food/2023/06/21/titos-vodka-keg-price/70342132007/
Author(s): USA TODAY, Emily DeLetter, USA TODAY
Title: Tito's launches 'Tito's in a Big Can,' an empty cocktail keg listed at $200
have you ever wanted to own your very own keg of tito's handmade vodka? now, that dream can become a reality.the vodka brand is launching tito's in a big can, a $200, 128 ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co2 regulator, stainless steel spear, tap and handle and an extra set of o rings. it comes empty, but tito's said it can be filled with their own vodka or with anything else.tito's is also releasing tito's in any can, a smaller insulated can cooler for $30 that also functions as a refillable cup. it can fit both 16 ounce and 12 ounce cans, and converts into a refillable pint tumbler with a leak proof lid."rebel against canformity," the product's description reads. "because we don't make canned cocktail

### Remove returns and tabs

#### Create function

In [34]:
rex_tabs = rex.compile(r'[\t]+')
rex_returns = rex.compile(r'[\n]+')
rex_errant_returns = rex.compile(r'[\n][\s]+[\n]')
rex_numbers = rex.compile(r'[\d]+')
rex_multispaces = rex.compile(r'[ ]{2,}')
rex_periods = rex.compile(r'[\S]\.')

def remove_returns(text):
    repl_text = rex_tabs.sub(r' ', text)
    repl_text = rex_returns.sub(r'\n', repl_text)
    repl_text = rex_errant_returns.sub(r'\n', repl_text)
    repl_text = rex_numbers.sub(r' ', repl_text)
    repl_text = rex_multispaces.sub(r' ', repl_text)
    #repl_text = rex_periods.sub(lambda match: match.group(0) + ' ', repl_text)
    #repl_text = rex_periods.sub(r'\. ', repl_text)
    return repl_text

#### Call function

Apply custom function to entire text column of the dataframe.

In [35]:
slct_tbl_full_df03['w_one_return'] = slct_tbl_full_df03['urls_removed'].apply(remove_returns)

In [36]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for s in source_name_lst:
    df01 = slct_tbl_full_df03.loc[slct_tbl_full_df03['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['w_one_return'][c], '\n')
        except:
            print(f'Skip {c}')

(36418, 22)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,author_lower_p_join,norm,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,deletter usa today emily,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa ashford today joy,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\nralph sampson breaks down iconic boston celt...


USA Today article #1
URL: https://www.usatoday.com/story/money/food/2023/06/21/titos-vodka-keg-price/70342132007/
Author(s): USA TODAY, Emily DeLetter, USA TODAY
Title: Tito's launches 'Tito's in a Big Can,' an empty cocktail keg listed at $200
have you ever wanted to own your very own keg of tito's handmade vodka? now, that dream can become a reality.the vodka brand is launching tito's in a big can, a $ , ounce stainless steel mini cocktail keg. the keg includes an airtight cap, a co regulator, stainless steel spear, tap and handle and an extra set of o rings. it comes empty, but tito's said it can be filled with their own vodka or with anything else.tito's is also releasing tito's in any can, a smaller insulated can cooler for $ that also functions as a refillable cup. it can fit both ounce and ounce cans, and converts into a refillable pint tumbler with a leak proof lid."rebel against canformity," the product's description reads. "because we don't make canned cocktails, pre mixes, o

### Calculate concentration ratio of each set of corpora

### KWIC

In [37]:
def kwic(doc_series, keyword, window=35, print_samples=5):
    '''Search article text for keywords in context (KWIC)'''
    def add_kwic(text):
        kwic_list.extend(keyword_in_context(doc=text,
                                            keyword=keyword,
                                            ignore_case=True,
                                            window_width=window))
    kwic_list = []
    doc_series.map(add_kwic)
    
    if print_samples is None or print_samples==0:
        return kwic_list
    else:
        k = min(print_samples, len(kwic_list))
        print(f"{k} random samples out of {len(kwic_list)}" + \
              f" contexts for '{keyword}':")
        for sample in random.sample(list(kwic_list), k):
            display(sample[0] + sample[1] + sample[2])
            print('\n')

In [38]:
#kwic(slct_tbl_full_df03['article_text'], 'amp', window=150, print_samples=120)

kwic(slct_tbl_full_df03['rex_trimmed'],
     "terms and conditions",
     window=200,
     print_samples=500)

317 random samples out of 317 contexts for 'terms and conditions':


'd the miami heat at kaseya center on june 09, 2023 in miami, florida. note to user: user expressly acknowledges and agrees that, by downloading and or using this photograph, user is consenting to the terms and conditions of the getty images license agreement. (photo by megan briggs/getty images)more \n\n\n\n\n\n share\n\n\n\n\n\t\t\tshare this\t\t\n\n\nimage\n\n\ngallery\n\n\n\n\n \n\t\t\t\t\ton facebook\t\t\t\t\n\n\n\n \n\t\t\t\t\ton twitter\t\t\t\t\n\n\n\n \n\t\t\t\t\tvia te'

'entry gains 1+ yard! with code nypnfl\t\t\n\n\n\t\t\t\tget bonus\t\t\t\n\n\n\n\t\t\tnew users only, 18+. ak, ar, ca, fl, ga, il, ks, ky, ma, mi, mn, ne, nh, nm, nc, nd, ok, or, ri, sc, sd, tx, ut, wi, wy, dc only. full terms and conditions apply. \t\t\n\n\n\n\n\n\n\n\t\t\tget $250 deposit match + free concert tickets with code nypbonus\t\t\n\n\n\t\t\t\tget bonus\t\t\t\n\n\n\n\t\t\tfirst time deposits only. 18+. must be located in eligible states (ak, ar, ca, co, fl, '

'st time deposits only. 18+. must be located in eligible states (ak, ar, ca, fl, ga, il, ks, ky, ma, mn, ne, nm, nc, nd, ok, or, ri, sc, sd, tx, ut, vt, wi, wy, dc) for the duration of a contest. full terms and conditions apply. \t\t\n\n\n\n\n\n\n\n\t\t\tinstant deposit match up to $100 on first deposit with code nypbonus\t\t\n\n\n\t\t\t\tget bonus\t\t\t\n\n\n\n\t\t\tmust be at least 18 years of age and within applicable state (al, ak, ar, ca, co, f'

'l 16, 2023 in milwaukee, wisconsin. miami defeated milwaukee 130-117. note to user: user expressly acknowledges and agrees that, by downloading and or using this photograph, user is consenting to the terms and conditions of the getty images license agreement. (photo by stacy revere/getty images)getty images\n"we\'re not trying to build a roster like we have in the past." jon horst made that clear with his two second-ro'

...

"table trust's portfolio. if jim has talked about a stock on cnbc tv, he waits 72 hours after issuing the trade alert before executing the trade. the above investing club information is subject to our terms and conditions and privacy policy , together with our disclaimer . no fiduciary obligation or duty exists, or is created, by virtue of your receipt of any information provided in connection with the investing club."

"table trust's portfolio. if jim has talked about a stock on cnbc tv, he waits 72 hours after issuing the trade alert before executing the trade. the above investing club information is subject to our terms and conditions and privacy policy , together with our disclaimer . no fiduciary obligation or duty exists, or is created, by virtue of your receipt of any information provided in connection with the investing club."

' game at golden 1 center on march 16, 2022 in sacramento, california. note to user: user expressly acknowledges and agrees that, by downloading and or using this photograph, user is consenting to the terms and conditions of the getty images license agreement. (photo by thearon w. henderson/getty images)getty images\nthe milwaukee bucks face a complex offseason that will challenge their front office to balance short-te'

'llowing her life where she\'s played by salma hayek. this leads to a spiral of joan doing obscene things to try to get hayek to stop portraying her, but this leads down a rabbit hole where because of "terms and conditions" both joan and hayek (who is literally playing herself) have signed their life and likeness right away with no legal recourse. then we get into deeply, deeply, meta territory when the entire project '

'entry gains 1+ yard! with code nypnfl\t\t\n\n\n\t\t\t\tget bonus\t\t\t\n\n\n\n\t\t\tnew users only, 18+. ak, ar, ca, fl, ga, il, ks, ky, ma, mi, mn, ne, nh, nm, nc, nd, ok, or, ri, sc, sd, tx, ut, wi, wy, dc only. full terms and conditions apply. \t\t\n\n\n\n\n\n\n\n\t\t\tget $250 deposit match + free concert tickets with code nypbonus\t\t\n\n\n\t\t\t\tget bonus\t\t\t\n\n\n\n\t\t\tfirst time deposits only. 18+. must be located in eligible states (ak, ar, ca, co, fl, '

### Sandbox for testing application of complex regular expressions to clean data

In [39]:
text2find_rex02 = rex.compile(r'http[s]?:[\/]+[\S]*\s')
text2find_rex02 = rex.compile(r'(?:http|https)*[:|\/]*(www)?[\S]*\b')
text2find_rex02 = rex.compile(r'(?:http|https|www)+[\.|\S]+\b')

test_lst02 = []

def test02(text):
    if text2find_rex02.findall(text) == []:
        pass
    else:
        test_lst02.append(text2find_rex02.findall(text))
    
slct_tbl_full_df03['emoji_split'].apply(test02)

print(len(test_lst02[:]))
test_lst03 = list(itertools.chain.from_iterable(test_lst02))
test_st01 = set(test_lst03)
test_lst03 = list(test_st01)
print(len(sorted(test_lst03)))
display(sorted(test_lst03[:1000]))

6933
4884


['http://apimagesblog.com',
 'http://twitter.com/ap_imagesap',
 'http://twitter.com/timelywriter',
 'http://www.twitter.com/etuckerap___more',
 'https://aggieswire.usatoday.com/2023/06/14/texas',
 'https://aggieswire.usatoday.com/2023/06/18/texas',
 'https://aggieswire.usatoday.com/2023/06/20/texas',
 'https://aggieswire.usatoday.com/2023/06/22/2024',
 'https://aggieswire.usatoday.com/2023/06/24/2024',
 'https://aggieswire.usatoday.com/2023/06/24/texas',
 'https://aggieswire.usatoday.com/2023/06/25/2024',
 'https://aggieswire.usatoday.com/2023/06/29/texas',
 'https://aggieswire.usatoday.com/2023/06/30/texas',
 'https://apnews.com/hub/africa',
 'https://apnews.com/hub/climate',
 'https://apnews.com/hub/college',
 'https://apnews.com/hub/death',
 'https://apnews.com/hub/donald',
 'https://apnews.com/hub/election',
 'https://apnews.com/hub/mlb',
 'https://apnews.com/hub/philanthropy',
 'https://apnews.com/hub/stanley',
 'https://apnews.com/hub/tennis',
 'https://apnews.com/hub/venezuela',

In [40]:
display(sorted(test_lst03[-1000:]))

['http://selfdeterminationtheory.org/sdt/documents/2008_grant_jap_tasksignificance.pdf',
 'http://t.co/qojrjsxq',
 'http://t.co/vxw2kgrow2',
 'http://twitter.com/anthonyted',
 'http://twitter.com/cappelletti7',
 'http://twitter.com/litalie',
 'http://twitter.com/litalie___this',
 'http://twitter.com/megkinnardap',
 'http://www.1800gambler.net',
 'http://www.twitter.com/etuckerap__associated',
 'https://aggieswire.usatoday.com/2023/06/13/texas',
 'https://aggieswire.usatoday.com/2023/06/15/2024',
 'https://aggieswire.usatoday.com/2023/06/18/2024',
 'https://aggieswire.usatoday.com/2023/06/19/texas',
 'https://aggieswire.usatoday.com/2023/06/24/terry',
 'https://aggieswire.usatoday.com/2023/06/28/ryan',
 'https://apnews.com/entertainment',
 'https://apnews.com/hub/apf',
 'https://apnews.com/hub/lgbtq',
 'https://apnews.com/hub/nfl',
 'https://apnews.com/hub/us',
 'https://auburnwire.usatoday.com/2023/06/14/key',
 'https://auburnwire.usatoday.com/2023/06/15/auburn',
 'https://auburnwire.u

### Lemmatization using spaCY

#### Display globally unqiue tokens on lemmas

In [41]:
#uniq_tok(df_col=slct_tbl_full_df03['lemma'])

### Split text

#### Apply

In [42]:
slct_tbl_full_df03['split'] = slct_tbl_full_df03['w_one_return']\
.apply(str.split)

In [43]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    try:
        print(slct_tbl_full_df03['split'][c], '\n')
    except:
        print(f'Skip {c}')

(36418, 23)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,norm,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return,split
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire to

['have', 'you', 'ever', 'wanted', 'to', 'own', 'your', 'very', 'own', 'keg', 'of', "tito's", 'handmade', 'vodka?', 'now,', 'that', 'dream', 'can', 'become', 'a', 'reality.the', 'vodka', 'brand', 'is', 'launching', "tito's", 'in', 'a', 'big', 'can,', 'a', '$', ',', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg.', 'the', 'keg', 'includes', 'an', 'airtight', 'cap,', 'a', 'co', 'regulator,', 'stainless', 'steel', 'spear,', 'tap', 'and', 'handle', 'and', 'an', 'extra', 'set', 'of', 'o', 'rings.', 'it', 'comes', 'empty,', 'but', "tito's", 'said', 'it', 'can', 'be', 'filled', 'with', 'their', 'own', 'vodka', 'or', 'with', 'anything', "else.tito's", 'is', 'also', 'releasing', "tito's", 'in', 'any', 'can,', 'a', 'smaller', 'insulated', 'can', 'cooler', 'for', '$', 'that', 'also', 'functions', 'as', 'a', 'refillable', 'cup.', 'it', 'can', 'fit', 'both', 'ounce', 'and', 'ounce', 'cans,', 'and', 'converts', 'into', 'a', 'refillable', 'pint', 'tumbler', 'with', 'a', 'leak', 'proof', 'lid.

#### Display globablly unqiue tokens on first split

In [44]:
uniq_tok(df_col=slct_tbl_full_df03['split'])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '!!!!!!!!',
 '!!!!!!!!!!!!!!',
 '!!!!"',
 '!!!!)',
 '!!!"',
 '!!!)',
 '!!!,"',
 '!!"',
 '!!"—martha',
 '!!#nbadraft',
 '!!)',
 '!!get',
 '!!️',
 '!!️!!️',
 '!!️@anniehanson_@coachsimmonssc@coachlukehuard',
 '!!️after',
 '!!️sensitive',
 '!"',
 '!"—desireedallas',
 '!"—shelley',
 '!#pflregularseason',
 "!'",
 '!\'"',
 "!''",
 '!)',
 '!),',
 '!).',
 '!)promising',
 '!+',
 '!==a.data["datawrapper',
 '!?',
 '!?"yes,',
 '!@chadsimmons_',
 '!@coachcaddy',
 '!@polonupe',
 '!check',
 '!fanduel',
 '!former',
 '!function(){"use',
 '!i',
 '!intiminapromising',
 '!promising',
 '!shop',
 '!sportsbook',
 '!targetpromising',
 '!targetthe',
 "!they're",
 '!—',
 '"',
 '"!!!!!!!!!"',
 '"!!"',
 '""',
 '""[it]',
 '""and',
 '""but',
 '""by',
 '""ezra',
 '""fiery',
 '""i',
 '""i\'ve',
 '""it',
 '""many',
 '""speedy',
 '""stop',
 '""ten',
 '""the',
 '""we',
 '""yes,',
 '"#',
 '"#acting,"',
 '"#advertisingfail."',
 '"#americanpridemonth."',
 '"#asmr',
 '"#awarenessmatt

### Remove stop words

In [45]:
sw = stopwords.words("english")

# Add additional stop words
sw.extend(['',
           '️',
           'arent',
           'cannot',
           'cant',
           'couldnt',
           'couldve',
           'didnt',
           'doesnt',
           'dont',
           'hadnt',
           'hasnt',
           'havent',
           'hes',
           'im',
           "i'm",
           'isnt',
           'it’s',
           'ive',
           '𝚘𝚏',
           'mightnt',
           'mustnt',
           'neednt',
           'shant',
           'shes',
           'shouldnt',
           'shouldve',
           'thatll',
           'theyll',
           'theyve',
           'wasnt',
           'werent',
           'whats',
           'weve',
           'wont',
           'wouldnt',
           'wouldve',
           'yall',
           'youd',
           'youll',
           'youre',
           'youve',
           "we'll",
           "you’re",
           "you’ve",
           "you’ll",
           "you’d",
           "she’s",
           "it’s",
           "that’ll",
           "don’t",
           "should’ve",
           "aren’t",
           "couldn’t",
           "didn’t",
           "doesn’t",
           "hadn’t",
           "hasn’t",
           "haven’t",
           "isn’t",
           "mightn’t",
           "mustn’t",
           "needn’t",
           "shan’t",
           "shouldn’t",
           "wasn’t",
           "weren’t",
           "won’t",
           "wouldn’t",
           "i’m",
           "we’ll",
           'said',
           'told',
           'according',
           'reporting',
           'reported',
           'statement',
           'spoke',
           'next',
           'though',
           'often',
           'story',
           'updated',
           'additional',
           'developments',
           'follow',
           'published',
           'com',
           'sunday',
           'monday',
           'tuesday',
           'wednesday',
           'thursday',
           'friday',
           'saturday',
           'january',
           'february',
           'march',
           'april',
           'may',
           'june',
           'july',
           'august',
           'september',
           'october',
           'november',
           'december',
          ])

print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Create function

In [46]:
def sw_remover(tokens):
    return [t for t in tokens if t.lower() not in sw]

#### Call function

Apply custom function to entire text column of the dataframe.

In [47]:
slct_tbl_full_df03['no_sw'] = slct_tbl_full_df03['split'].apply(sw_remover)

In [48]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    print(slct_tbl_full_df03['no_sw'][c])

(36418, 24)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,norm_author_lower_p_join,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return,split,no_sw
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, tito's, handmade, vodka?, ..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\nralph sampson breaks down i

['ever', 'wanted', 'keg', "tito's", 'handmade', 'vodka?', 'now,', 'dream', 'become', 'reality.the', 'vodka', 'brand', 'launching', "tito's", 'big', 'can,', '$', ',', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg.', 'keg', 'includes', 'airtight', 'cap,', 'co', 'regulator,', 'stainless', 'steel', 'spear,', 'tap', 'handle', 'extra', 'set', 'rings.', 'comes', 'empty,', "tito's", 'filled', 'vodka', 'anything', "else.tito's", 'also', 'releasing', "tito's", 'can,', 'smaller', 'insulated', 'cooler', '$', 'also', 'functions', 'refillable', 'cup.', 'fit', 'ounce', 'ounce', 'cans,', 'converts', 'refillable', 'pint', 'tumbler', 'leak', 'proof', 'lid."rebel', 'canformity,"', "product's", 'description', 'reads.', '"because', 'make', 'canned', 'cocktails,', 'pre', 'mixes,', 'seltzers.', 'do."', 'best:which', 'fast', 'food', 'restaurants', 'reign', 'supreme?', 'cast', 'votes', 'nowwhat', 'americans', 'drinking', 'summer?tequila,', 'modelo,', 'canned', 'cocktails', 'non', 'alcoholic', 'drinks

#### Display no stop words

In [49]:
#uniq_tok(df_col=slct_tbl_full_df03['no_sw'])

### Rejoin semi-processed tokens

In [50]:
slct_tbl_full_df03['no_sw_join'] = slct_tbl_full_df03['no_sw'].apply(" ".join)

In [51]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    print(slct_tbl_full_df03['no_sw_join'][c])

(36418, 25)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,rex_trimmed,trimmed,trimmed_norm_author_lower_p_join,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return,split,no_sw,no_sw_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, tito's, handmade, vodka?, ...","ever wanted keg tito's handmade vodka? now, dr..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph 

ever wanted keg tito's handmade vodka? now, dream become reality.the vodka brand launching tito's big can, $ , ounce stainless steel mini cocktail keg. keg includes airtight cap, co regulator, stainless steel spear, tap handle extra set rings. comes empty, tito's filled vodka anything else.tito's also releasing tito's can, smaller insulated cooler $ also functions refillable cup. fit ounce ounce cans, converts refillable pint tumbler leak proof lid."rebel canformity," product's description reads. "because make canned cocktails, pre mixes, seltzers. do." best:which fast food restaurants reign supreme? cast votes nowwhat americans drinking summer?tequila, modelo, canned cocktails non alcoholic drinksboth products available limited time purchased online love, tito's physical store austin, texas. release, tito's net proceeds cans go toward customer's choice nonprofit brand teamed with.


### Remove punctuation

In [52]:
punctuation = set(punctuation) # speeds up comparison
#print(punctuation)

# Add special hyphen mark
tw_punct = punctuation - {"#"}
#print(tw_punct)

# Remove hash and at symbols for later capture of hashtag info
tw_punct = tw_punct - {"@"}
tw_punct = tw_punct - {"-"}
tw_punct = tw_punct - {"."}
tw_punct = tw_punct - {"?"}
tw_punct = tw_punct - {"!"}
#tw_punct = tw_punct - {"/"}
tw_punct.add("’")
tw_punct.add("‘")
tw_punct.add("”")
tw_punct.add("“")
tw_punct.add("…")
tw_punct.add("—")
tw_punct.add("...")
tw_punct.add("€")
tw_punct.add("±")
tw_punct.add("£")
tw_punct.add("¡")
tw_punct.add("§")
tw_punct.add("⦿")

print(tw_punct)

{'¡', '_', '"', '{', '&', '`', ';', '£', ':', '=', '”', '…', '~', '±', ']', ',', '+', '/', '“', ')', '⦿', '^', '[', '\\', '}', '>', '‘', '...', '€', '<', '%', '(', '*', '|', '—', '$', '§', '’', "'"}


#### Create function

In [53]:
def remove_punctuation(text,
                       punct_set=tw_punct): 
    return("".join([ch for ch in text if ch not in punct_set]))

#### Call function

Apply custom function to entire text column of the dataframe.

In [54]:
slct_tbl_full_df03['no_sw_join_no_punc'] = slct_tbl_full_df03['no_sw_join']\
.apply(remove_punctuation, punct_set=tw_punct)

slct_tbl_full_df03['no_sw_join_no_punc_urls_removed_norm_author_lower_p_join'] = slct_tbl_full_df03['urls_removed_norm_author_lower_p_join']\
.apply(remove_punctuation, punct_set=tw_punct)

slct_tbl_full_df03['split_no_sw_join_no_punc_urls_removed_norm_author_lower_p_join'] = slct_tbl_full_df03['no_sw_join_no_punc_urls_removed_norm_author_lower_p_join']\
.apply(str.split)

In [55]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

(36418, 28)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,emoji_split,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return,split,no_sw,no_sw_join,no_sw_join_no_punc,no_sw_join_no_punc_urls_removed_norm_author_lower_p_join,split_no_sw_join_no_punc_urls_removed_norm_author_lower_p_join
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, tito's, handmade, vodka?, ...","ever wanted keg tito's handmade vodka? now, dr...",ever wanted keg titos handmade vodka? now drea...,deletter usa today emily,"[deletter, usa, today, emily]"
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,five months julian sands went missing solo hik...,usa ashford today joy,"[usa, ashford, today, joy]"
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,four star running back picks michigan state un...,usa today wire tar heels,"[usa, today, wire, tar, heels]"
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,roll usa today wire tide,"[roll, usa, today, wire, tide]"
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, bre

In [56]:
for s in source_name_lst:
    df01 = slct_tbl_full_df03.loc[slct_tbl_full_df03['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['no_sw_join_no_punc'][c], '\n')
        except:
            print(f'Skip {c}')

USA Today article #1
URL: https://www.usatoday.com/story/money/food/2023/06/21/titos-vodka-keg-price/70342132007/
Author(s): USA TODAY, Emily DeLetter, USA TODAY
Title: Tito's launches 'Tito's in a Big Can,' an empty cocktail keg listed at $200
ever wanted keg titos handmade vodka? now dream become reality.the vodka brand launching titos big can   ounce stainless steel mini cocktail keg. keg includes airtight cap co regulator stainless steel spear tap handle extra set rings. comes empty titos filled vodka anything else.titos also releasing titos can smaller insulated cooler  also functions refillable cup. fit ounce ounce cans converts refillable pint tumbler leak proof lid.rebel canformity products description reads. because make canned cocktails pre mixes seltzers. do. bestwhich fast food restaurants reign supreme? cast votes nowwhat americans drinking summer?tequila modelo canned cocktails non alcoholic drinksboth products available limited time purchased online love titos physical s

In [57]:
uniq_tok(slct_tbl_full_df03['split_no_sw_join_no_punc_urls_removed_norm_author_lower_p_join'])

7398
['1', '10best', '10best.com', '19th', '2', '200', '5', 'a', 'a.', 'a.j.', 'aaditya', 'aamer', 'aarian', 'aaron', 'abad', 'abakar', 'abbas', 'abbey', 'abbi', 'abbott', 'abbruzzese', 'abby', 'abc', 'abdallat', 'abdelbary', 'abdo', 'abdul', 'abdulaziz', 'abdulla', 'abdulrahman', 'abeer', 'abel', 'abeywardena', 'abha', 'abhay', 'abhimanyu', 'abhinav', 'abigail', 'abitter@businessinsider.com', 'ables', 'abrahamson', 'abrams', 'abuelsamid', 'abyzov', 'accettura', 'acevedo', 'acharya', 'achenbach', 'achin', 'achmad', 'achuthanand', 'ackerman', 'ackermann', 'acree', 'action', 'ada', 'adam', 'adams', 'adamson', 'adaora', 'adavis@insider.com', 'addison', 'adebayo', 'adel', 'adele', 'adeoye', 'adeshola', 'adgate', 'adi', 'adiaznyp', 'adib', 'adiel', 'adiri', 'aditya', 'adkisson', 'adrian', 'adriana', 'adrianna', 'adrien', 'adrienne', 'adunola', 'advani', 'advertiser', 'advisor', 'advisors', 'adyen', 'aerospace', 'afdhel', 'affairs', 'afp', 'agarwal', 'aggarwal', 'aggies', 'agiesta', 'ago', '

['1',
 '10best',
 '10best.com',
 '19th',
 '2',
 '200',
 '5',
 'a',
 'a.',
 'a.j.',
 'aaditya',
 'aamer',
 'aarian',
 'aaron',
 'abad',
 'abakar',
 'abbas',
 'abbey',
 'abbi',
 'abbott',
 'abbruzzese',
 'abby',
 'abc',
 'abdallat',
 'abdelbary',
 'abdo',
 'abdul',
 'abdulaziz',
 'abdulla',
 'abdulrahman',
 'abeer',
 'abel',
 'abeywardena',
 'abha',
 'abhay',
 'abhimanyu',
 'abhinav',
 'abigail',
 'abitter@businessinsider.com',
 'ables',
 'abrahamson',
 'abrams',
 'abuelsamid',
 'abyzov',
 'accettura',
 'acevedo',
 'acharya',
 'achenbach',
 'achin',
 'achmad',
 'achuthanand',
 'ackerman',
 'ackermann',
 'acree',
 'action',
 'ada',
 'adam',
 'adams',
 'adamson',
 'adaora',
 'adavis@insider.com',
 'addison',
 'adebayo',
 'adel',
 'adele',
 'adeoye',
 'adeshola',
 'adgate',
 'adi',
 'adiaznyp',
 'adib',
 'adiel',
 'adiri',
 'aditya',
 'adkisson',
 'adrian',
 'adriana',
 'adrianna',
 'adrien',
 'adrienne',
 'adunola',
 'advani',
 'advertiser',
 'advisor',
 'advisors',
 'adyen',
 'aerospace',

In [58]:
text2find_rex02 = rex.compile(r'\b[\S]+[\.|\?|!]+[\S]+\b')

test_lst02 = []

def test02(text):
    if text2find_rex02.findall(text) == []:
        pass
    else:
        test_lst02.append(text2find_rex02.findall(text))
    
slct_tbl_full_df03['no_sw_join_no_punc'].apply(test02)

print(len(test_lst02[:]))
test_lst03 = list(itertools.chain.from_iterable(test_lst02))
test_st01 = set(test_lst03)
test_lst03 = list(test_st01)
print(len(sorted(test_lst03)))
display(sorted(test_lst03))

26312
103375


['a!c',
 'a...byjim',
 'a...cage',
 'a...certain',
 'a...diablo',
 'a...fight',
 'a...las',
 'a...light',
 'a...reaction',
 'a...read',
 'a...strangely',
 'a.a.get',
 'a.age',
 'a.b',
 'a.b.get',
 'a.betterbrand',
 'a.but',
 'a.c',
 'a.c.h',
 'a.c.k',
 'a.charles',
 'a.child',
 'a.com',
 'a.consumerget',
 'a.d',
 'a.d.h.d',
 'a.datadatawrapper',
 'a.e',
 'a.f.get',
 'a.g',
 'a.get',
 'a.getty',
 'a.h.beard',
 'a.i',
 'a.i.ai',
 'a.i.business',
 'a.i.by',
 'a.i.digital',
 'a.i.google',
 'a.i.mad',
 'a.i.nathan',
 'a.i.nvidias',
 'a.i.s',
 'a.i.squawk',
 'a.i.watch',
 'a.i.while',
 'a.i.zoom',
 'a.j',
 'a.jesdani',
 'a.k.a',
 'a.kitchen',
 'a.l',
 'a.l.i.get',
 'a.m',
 'a.m..amazonpromising',
 'a.m.adamolekun',
 'a.m.amazonpromising',
 'a.m.ap',
 'a.m.b',
 'a.m.citizen',
 'a.m.facebook',
 'a.m.in',
 'a.m.miles',
 'a.m.p.a.s',
 'a.m.paul',
 'a.m.peacocksecond',
 'a.m.putin',
 'a.m.share',
 'a.m.the',
 'a.m.this',
 'a.m.ukraine',
 'a.m.usa',
 'a.m.vulcan',
 'a.m.wagner',
 'a.p.f',
 'a.p.ge

### Tokenize

In [59]:
slct_tbl_full_df03['no_sw_join_no_punc_tok'] = slct_tbl_full_df03['no_sw_join_no_punc'].apply(str.split)

print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    print(slct_tbl_full_df03['no_sw_join_no_punc_tok'][c], '\n')

(36418, 29)


,text_id,source_name,author,title,url,publish_date,article_text,content,lower,author_source,...,urls_removed,urls_removed_norm_author_lower_p_join,w_one_return,split,no_sw,no_sw_join,no_sw_join_no_punc,no_sw_join_no_punc_urls_removed_norm_author_lower_p_join,split_no_sw_join_no_punc_urls_removed_norm_author_lower_p_join,no_sw_join_no_punc_tok
0,2,USA Today,"USA TODAY, Emily DeLetter, USA TODAY","Tito's launches 'Tito's in a Big Can,' an empt...",https://www.usatoday.com/story/money/food/2023...,2023-06-21T17:37:40Z,Have you ever wanted to own your very own keg ...,Have you ever wanted to own your very own keg ...,have you ever wanted to own your very own keg ...,"USA Today USA TODAY, Emily DeLetter, USA TODAY",...,have you ever wanted to own your very own keg ...,deletter usa today emily,have you ever wanted to own your very own keg ...,"[have, you, ever, wanted, to, own, your, very,...","[ever, wanted, keg, tito's, handmade, vodka?, ...","ever wanted keg tito's handmade vodka? now, dr...",ever wanted keg titos handmade vodka? now drea...,deletter usa today emily,"[deletter, usa, today, emily]","[ever, wanted, keg, titos, handmade, vodka?, n..."
1,3,USA Today,"USA TODAY, Joy Ashford, USA TODAY",Search for missing actor Julian Sands continue...,https://www.usatoday.com/story/entertainment/c...,2023-06-20T17:36:09Z,Over five months after Julian Sands went missi...,Over five months after Julian Sandswent missin...,over five months after julian sands went missi...,"USA Today USA TODAY, Joy Ashford, USA TODAY",...,over five months after julian sands went missi...,usa ashford today joy,over five months after julian sands went missi...,"[over, five, months, after, julian, sands, wen...","[five, months, julian, sands, went, missing, s...",five months julian sands went missing solo hik...,five months julian sands went missing solo hik...,usa ashford today joy,"[usa, ashford, today, joy]","[five, months, julian, sands, went, missing, s..."
2,5,USA Today,Tar Heels Wire,Four star running back picks Michigan State ov...,https://tarheelswire.usatoday.com/2023/06/24/f...,2023-06-25T03:51:10Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFour star running ...,Mack Brown and the UNC football program have b...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,USA Today Tar Heels Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nfour star running ...,usa today wire tar heels,\nfour star running back picks michigan state ...,"[four, star, running, back, picks, michigan, s...","[four, star, running, back, picks, michigan, s...",four star running back picks michigan state un...,four star running back picks michigan state un...,usa today wire tar heels,"[usa, today, wire, tar, heels]","[four, star, running, back, picks, michigan, s..."
3,6,USA Today,Roll Tide Wire,Alabama center Charles Bediako signs one-year ...,https://rolltidewire.usatoday.com/2023/06/23/a...,2023-06-23T21:29:24Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nAlabama center Cha...,Alabama center Charles Bediako was signed to a...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,USA Today Roll Tide Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nalabama center cha...,roll usa today wire tide,\nalabama center charles bediako signs one yea...,"[alabama, center, charles, bediako, signs, one...","[alabama, center, charles, bediako, signs, one...",alabama center charles bediako signs one year ...,alabama center charles bediako signs one year ...,roll usa today wire tide,"[roll, usa, today, wire, tide]","[alabama, center, charles, bediako, signs, one..."
4,7,USA Today,Celtics Wire,Ralph Sampson breaks down iconic Boston Celtic...,https://celticswire.usatoday.com/2023/06/23/nb...,2023-06-23T11:00:41Z,\n\n\n\n\n\n\n\n\n\n\n\n\n\nRalph Sampson brea...,It was one of the most memorable moments in NB...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,USA Today Celtics Wire,...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nralph sampson brea...,usa wire today celtics,\nralph sampson breaks down iconic boston celt...,"[ralph, sampson, breaks, down, iconic, boston,...",

['ever', 'wanted', 'keg', 'titos', 'handmade', 'vodka?', 'now', 'dream', 'become', 'reality.the', 'vodka', 'brand', 'launching', 'titos', 'big', 'can', 'ounce', 'stainless', 'steel', 'mini', 'cocktail', 'keg.', 'keg', 'includes', 'airtight', 'cap', 'co', 'regulator', 'stainless', 'steel', 'spear', 'tap', 'handle', 'extra', 'set', 'rings.', 'comes', 'empty', 'titos', 'filled', 'vodka', 'anything', 'else.titos', 'also', 'releasing', 'titos', 'can', 'smaller', 'insulated', 'cooler', 'also', 'functions', 'refillable', 'cup.', 'fit', 'ounce', 'ounce', 'cans', 'converts', 'refillable', 'pint', 'tumbler', 'leak', 'proof', 'lid.rebel', 'canformity', 'products', 'description', 'reads.', 'because', 'make', 'canned', 'cocktails', 'pre', 'mixes', 'seltzers.', 'do.', 'bestwhich', 'fast', 'food', 'restaurants', 'reign', 'supreme?', 'cast', 'votes', 'nowwhat', 'americans', 'drinking', 'summer?tequila', 'modelo', 'canned', 'cocktails', 'non', 'alcoholic', 'drinksboth', 'products', 'available', 'limite

#### Display globally unqiue tokens on final tokens

In [60]:
uniq_tok(df_col=slct_tbl_full_df03['no_sw_join_no_punc_tok'])

389463


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '!!!!!!!!',
 '!!!!!!!!!',
 '!!!!!!!!!!!!!!',
 '!!!!.',
 '!!!.',
 '!!#nbadraft',
 '!!.',
 '!!get',
 '!!martha',
 '!!️',
 '!!️!!️',
 '!!️@anniehanson@coachsimmonssc@coachlukehuard',
 '!!️after',
 '!!️sensitive',
 '!#pflregularseason',
 '!.',
 '!?',
 '!?yes',
 '!@chadsimmons',
 '!@coachcaddy',
 '!@polonupe',
 '!a.datadatawrapper',
 '!check',
 '!desireedallas',
 '!even',
 '!fanduel',
 '!former',
 '!functionuse',
 '!i',
 '!intiminapromising',
 '!promising',
 '!shelley',
 '!shop',
 '!sportsbook',
 '!targetpromising',
 '!targetthe',
 '!theyre',
 '#',
 '#!',
 '###',
 '#@!',
 '#a',
 '#abortion',
 '#accessibility',
 '#accidentallywesanderson',
 '#accountant',
 '#acting',
 '#actonclimate',
 '#actor',
 '#ad.',
 '#adulting!',
 '#adultjuice',
 '#advertisingfail',
 '#advertisingfail.',
 '#adviceforgirls',
 '#aesthetic',
 '#aew',
 '#aewxnjpw',
 '#affirmativeaction.',
 '#afghanistan',
 '#ag',
 '#againstallodds',
 '#aggies',
 '#aggressive',
 '#agtg',
 '#agtg.',
 

### Remove `author` and `source_name` values

In [65]:
def auth_remover(tokens, other_column):
    return [t for t in tokens if t not in other_column]

slct_tbl_full_df03['no_sw_join_no_punc_no_authorsource'] = slct_tbl_full_df03.apply(lambda row: auth_remover(row['no_sw_join_no_punc_tok'],
                                                                                                             row['split_no_sw_join_no_punc_urls_removed_norm_author_lower_p_join']),
                                                                                    axis=1)

In [ ]:
print(slct_tbl_full_df03.shape)
display(slct_tbl_full_df03.head())

for c in range(0,1):
    print(slct_tbl_full_df03['no_sw_join_no_punc_tok'][c], '\n')

In [ ]:
slct_tbl_full_df03['no_sw_join_no_punc_no_authorsource_join'] = slct_tbl_full_df03['no_sw_join_no_punc_no_authorsource'].apply(" ".join)

### Count characters and words for initial review

In [ ]:
# Raw text character and word counts
slct_tbl_full_df03['char_cnt'] = slct_tbl_full_df03['no_sw_join_no_punc_no_authorsource_join']\
.progress_apply(len)
slct_tbl_full_df03['word_cnt'] = slct_tbl_full_df03['no_sw_join_no_punc_no_authorsource_join']\
.progress_apply(lambda x: len(x.split()))
display(slct_tbl_full_df03.head())

### Pipeline consolidation

#### Pipeline function

In [62]:
def prepare(text, pipeline):
    '''Run a pipeline of text processing transformers'''
    tokens = str(text)
    
    # Pull key and val from trans dictionaries
    for transformer in pipeline:
        trans = list(transformer.keys())[0]
        args = list(transformer.values())[0]
        #print(trans)
        #print(args)
        if args == None:
            #print(1)
            tokens = trans(tokens)
        else:
            #print('check99', trans, args)
            tokens = trans(tokens, args) 

    return(tokens)

#### `article_text` preprocessing pipeline - stop words removed

In [63]:
'''Set transformer pipeline 1:
Caseloading, normalization (using textacy), special ch removal,
split on whitespace, stop word removal, rejoin,
remove custom punctuation, tokenize
'''
transformers01 = [{str.lower: None},
                  {normalize: None},
                  {remove_rex: None},
                  {rex_replace: None},
                  {emoji_split: None},
                  {remove_url: None},
                  {remove_returns: None},
                  {str.split: None},
                  {sw_remover: None},
                  {" ".join: None},
                  {remove_punctuation: tw_punct},
                  {str.split: None},
                  {auth_remover: None},
                  {" ".join: None},
                 ]

# Apply transformers to pandas dataframe, w/ new col containing tokens
slct_tbl_full_df04['processed_text'] = slct_tbl_full_df04['article_text']\
.progress_apply(prepare,
                pipeline=transformers01)

slct_tbl_full_df04['processed_text_split'] = slct_tbl_full_df04['processed_text']\
.progress_apply(str.split)

slct_tbl_full_df04['num_tokens'] = slct_tbl_full_df04['processed_text_split']\
.map(len)

display(slct_tbl_full_df04.head())

  0%|           | 1/36418 [00:00<08:29, 71.42it/s]


TypeError: auth_remover() missing 1 required positional argument: 'axis'

In [ ]:
file_out_name01 = f'data_preprocessed_wo_sw_{today}.csv'
file_out_path01 = os.path.join(up1_dir, data_location, file_out_name01)
print(f'CSV file out 1 path: {file_out_path01}')

In [ ]:
slct_tbl_full_df04.to_csv(file_out_path01, index=False)

In [ ]:
for s in source_name_lst:
    df01 = slct_tbl_full_df04.loc[slct_tbl_full_df04['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['processed_text'][c], '\n')
        except:
            print(f'Skip {c}')

##### Display globally unqiue tokens on final tokens

In [ ]:
df04_t_len = len(uniq_tok(df_col=slct_tbl_full_df04['processed_text_split']))

In [ ]:
print(df04_t_len)

#### Display globally unqiue tokens on final tokens

In [ ]:
df03_t_len = len(uniq_tok(df_col=slct_tbl_full_df03['no_sw_join_no_punc_tok']))

In [ ]:
print(df03_t_len)

#### `article_text` preprocessing pipeline - stop words not removed

In [ ]:
'''Set transformer pipeline 1:
Caseloading, normalization (using textacy), special ch removal,
split on whitespace, stop word removal, rejoin,
remove custom punctuation, tokenize
'''
transformers02 = [{str.lower: None},
                  {normalize: None},
                  {remove_rex: None},
                  {rex_replace: None},
                  {emoji_split: None},
                  {remove_url: None},
                  {remove_returns: None},
                  {remove_punctuation: tw_punct},
                  {str.split: None},
                  {auth_remover: 1},
                  {" ".join: None},
                 ]

# Apply transformers to pandas dataframe, w/ new col containing tokens
slct_tbl_full_df05['processed_text'] = slct_tbl_full_df05['article_text']\
.progress_apply(prepare,
                pipeline=transformers02)

slct_tbl_full_df05['processed_text_split'] = slct_tbl_full_df05['processed_text']\
.progress_apply(str.split)

slct_tbl_full_df05['num_tokens'] = slct_tbl_full_df05['processed_text_split']\
.map(len)

display(slct_tbl_full_df05.head())

In [ ]:
file_out_name02 = f'data_preprocessed_w_sw_{today}.csv'
file_out_path02 = os.path.join(up1_dir, data_location, file_out_name02)
print(f'CSV file out 2 path: {file_out_path02}')

In [ ]:
slct_tbl_full_df05.to_csv(file_out_path02, index=False)

In [ ]:
for s in source_name_lst:
    df01 = slct_tbl_full_df05.loc[slct_tbl_full_df05['source_name'] == s]
    df01 = df01.reset_index()
    counter = 0
    for c in range(0,6):
        counter += 1
        print(f'{s} article #{counter}')
        try:
            print('URL:', df01['url'][c])
            print('Author(s):', df01['author'][c])
            print('Title:', df01['title'][c])
            print(df01['processed_text'][c], '\n')
        except:
            print(f'Skip {c}')

##### Display globally unqiue tokens on final tokens

In [ ]:
df05_t_len = len(uniq_tok(df_col=slct_tbl_full_df05['processed_text_split']))

In [ ]:
print(df05_t_len)

#### `article_text` preprocessing - w/ lemmatization

### Train/test split

### TF-IDF

In [ ]:
file_out_name01